## Approche dérangements

In [ ]:
import math

In [10]:
import operator as op
from functools import reduce
def ncr(n, r):
    r = min(r, n-r)
    if r == 0: return 1
    numer = reduce(op.mul, range(n, n-r, -1))
    denom = reduce(op.mul, range(1, r+1))
    return numer//denom

In [2]:
def nombre_derangements(n):
    fac_n = math.factorial(n)
    return sum((-1)**k * fac_n // math.factorial(k) for k in range(n+1))

In [8]:
n = 4
nb_der = [nombre_derangements(n-2*k) for k in range(n//2, -1, -1)]

In [35]:
def nombre_intransposements(n):
    nb_der = [nombre_derangements(n-2*k) for k in range(n//2+1)]
    return nb_der[0] - sum((-1)**(k+1) * ncr(n, 2*k) * math.factorial(2*k)//(2**k)//math.factorial(k) * nb_der[k] for k in range(1, n//2+1))

## Approche théorie des graphes

In [1]:
from collections import Counter, defaultdict
from itertools import product, chain

In [2]:
def arc(a, b):
    return tuple(sorted([a,b]))

In [3]:
def combine(arc1, arc2):
    if arc1[0] == arc2[0]:
        return arc(arc1[1], arc2[1])
    elif arc1[0] == arc2[1]:
        return arc(arc1[1], arc2[0])
    elif arc1[1] == arc2[0]:
        return arc(arc1[0], arc2[1])
    elif arc1[1] == arc2[1]:
        return arc(arc1[0], arc2[0])
    else:
        raise TypeError('arcs not combinable')

class ArcAggregator():
    def __init__(self, values):
        values = [arc(a,b) for a, b in values]
        self._arcs = frozenset(values)
        self._points = points = {}
        
        for a, b in values:
            points[a] = (a,b)
            points[b] = (a,b)
            
    def extend(self, a, b):
        points = self._points
        closed = 0
        res = []

        cur = arc(a,b)
        
        connected = frozenset(points[i] for i in cur if i in points)
        assert len(connected) <= 2
        
        for con in connected:
            if cur == con:
                # pour s'assurer qu'il n'y a plus d'autres chemins connectés
                cur = None
                closed += 1
            else:
                cur = combine(cur, con)
        
        res = self._arcs - connected
        if cur:
            res |= frozenset([cur])
        
        return ArcAggregator(res), closed
    
    def __repr__(self):
        return 'ArcSet(%s)' % (', '.join(repr(arc) for arc in self._arcs),)

In [27]:
size = 4
start_state = ({
    # ensemble d'arcs, ensemble de points
    (frozenset(), frozenset(['a0'])): Counter([0])
}, 0)

In [28]:
rows = 'abcdefghijklmnopqrstuvwxyz'
cols = '0123456789'

codes = {(i, j): rows[i] + cols[j] for i in range(size) for j in range(size)}

neighbours = {}

for i in range(size-1):
    for j in range(size-1):
        neighbours[rows[i]+cols[j]] = (rows[i+1]+cols[j], rows[i]+cols[j+1])

    neighbours[rows[i]+cols[size-1]] = (rows[i+1] + cols[size-1],)

for j in range(size-1):
    neighbours[rows[size-1]+ cols[j]] = (rows[size-1] + cols[j+1],)
    
neighbours[rows[size-1]+cols[size-1]] = []

diag_points = [frozenset(codes[j, i-j] for j in range(max(0, i-size+1), min(i+1, size))) for i in range(2*size-1)]

In [29]:
from IPython.core.debugger import Tracer

def next_level(prev):
    prev_state, prev_level = prev
    
    res = defaultdict(Counter)

    # les points de la nouvelle diagonale
    current_diag = diag_points[prev_level+1]
    
    for (endpoints, single_points), weights in prev_state.items():
        # on est certains que les points isolés seront raccordés
        
        # pour passer à la boucle suivante
        impossible = False
        
        new_paths = []
        seen = []
        for sp in single_points:
            neighs = neighbours[sp]
            # il faut nécessairement deux voisins pour les points isolés
            if len(neighs) < 2:
                impossible = True
                break
            for neigh in neighs:
                seen.append(neigh)
            new_paths.append(arc(*neighs))

        if impossible:
            continue
        
        # on fait la liste des points qu'il va falloir relier
        points = [point for path in endpoints for point in path]
        
        for new_endpoints, left, cycles in build_path(points, ArcAggregator(chain(new_paths, endpoints)), current_diag - frozenset(seen), 0):
            all_endpoints = frozenset(endpoint for path in new_endpoints._arcs for endpoint in path)
            assert len(all_endpoints & left) == 0
            assert (all_endpoints | left).issubset(current_diag)
            new_weights = Counter()
            for m in weights:
                new_weights[m+cycles] = weights[m]
                        
            res[new_endpoints._arcs, left] += new_weights
        
    return (res, prev_level+1)
        
def build_path(points, arcs, left, cycles):    
    if not points:
        yield (arcs, left, cycles)
        return
    
    point, *other_points = points
        
    for neigh in neighbours[point]:
        new_arcs, new_cycles = arcs.extend(point, neigh)
        yield from build_path(other_points, new_arcs, left - {neigh}, cycles + new_cycles)

In [30]:
states = [start_state]
cur = start_state

for _ in range(2*size-2):
    cur = next_level(cur)
    states.append(cur)

In [31]:
sum(states[-1][0][(frozenset(), frozenset())].values())

18

In [32]:
import networkx as nx

In [34]:
g = nx.grid_2d_graph(4,4)

In [37]:
%matplotlib inline

In [47]:
g.edges()

[((0, 1), (0, 0)),
 ((0, 1), (1, 1)),
 ((0, 1), (0, 2)),
 ((1, 2), (1, 1)),
 ((1, 2), (1, 3)),
 ((1, 2), (0, 2)),
 ((1, 2), (2, 2)),
 ((3, 2), (3, 1)),
 ((3, 2), (3, 3)),
 ((3, 2), (2, 2)),
 ((0, 0), (1, 0)),
 ((3, 3), (2, 3)),
 ((3, 0), (2, 0)),
 ((3, 0), (3, 1)),
 ((3, 1), (2, 1)),
 ((1, 1), (1, 0)),
 ((1, 1), (2, 1)),
 ((2, 1), (2, 0)),
 ((2, 1), (2, 2)),
 ((0, 2), (0, 3)),
 ((2, 0), (1, 0)),
 ((1, 3), (0, 3)),
 ((1, 3), (2, 3)),
 ((2, 3), (2, 2))]

## Approche problème de résolution de contraintes

In [27]:
# mes variables sont les nombre de 0 à 99, en ordre C
domaines = np.zeros(dtype=np.dtype('U6'), shape=(size, size))

# on change les coins
domaines[0,0] = '┌'
domaines[0,-1] = '┐'
domaines[-1,0] = '└'
domaines[-1,-1] = '┘'

# on change les bords
domaines[0,1:-1] = '─┌┐'
domaines[-1,1:-1] = '─└┘'
domaines[1:-1, 0] = '│┌└'
domaines[1:-1, -1] = '│┐┘'

# on change le milieu
domaines[1:-1, 1:-1] = directions

In [33]:
for row in domaines:
    print('[%s]' % (' '.join('%08s' % i for i in row),))

[       ┌      ─┌┐      ─┌┐      ─┌┐      ─┌┐        ┐]
[     │┌└   ─│┌┐└┘   ─│┌┐└┘   ─│┌┐└┘   ─│┌┐└┘      │┐┘]
[     │┌└   ─│┌┐└┘   ─│┌┐└┘   ─│┌┐└┘   ─│┌┐└┘      │┐┘]
[     │┌└   ─│┌┐└┘   ─│┌┐└┘   ─│┌┐└┘   ─│┌┐└┘      │┐┘]
[     │┌└   ─│┌┐└┘   ─│┌┐└┘   ─│┌┐└┘   ─│┌┐└┘      │┐┘]
[       └      ─└┘      ─└┘      ─└┘      ─└┘        ┘]


In [ ]:
propagate_functions = {}

def assign(grille, index, value):
    other_values = grille[index].replace(value, '')
    

def propagate_h(grille, index):
    i, j = index
    if i:
        value = grille[i-1, j].replace('│┐┘', '')
        if not value:
            return False
        if len(value) == 1:
            if not propagate(grille, (i-1, j), value):
                return False
        
    if i<size-1:
        value = grille[i+1, j].replace('│┌└', '')
        if not value:
            return False
        if len(value) == 1:
            if not propagate(grille, (i+1, j), value):
                return False

def propagate(grille, index, value):
    
